In [2]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
pcd = o3d.io.read_point_cloud("filtered.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
points = np.asarray(downpcd.points)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
normals = np.asarray(downpcd.normals)

In [23]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.optimize import least_squares

def calculate_density(points, radius=0.1):
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def residuals(params, points):
    x0, y0, z0, nx, ny, nz, width, height = params
    plane_normal = np.array([nx, ny, nz])
    plane_normal /= np.linalg.norm(plane_normal)

    # Project points onto the plane
    vector_to_points = points - np.array([x0, y0, z0])
    distance_to_plane = np.dot(vector_to_points, plane_normal)
    projection = vector_to_points - distance_to_plane[:, np.newaxis] * plane_normal
    projected_points = projection + np.array([x0, y0, z0])

    # Define two vectors to represent the rectangle's edges
    basis1 = np.cross(plane_normal, np.array([1, 0, 0]))
    if np.linalg.norm(basis1) < 1e-6:
        basis1 = np.cross(plane_normal, np.array([0, 1, 0]))
    basis1 /= np.linalg.norm(basis1)
    basis2 = np.cross(plane_normal, basis1)

    # Transform projected points into the rectangle's local coordinate frame
    relative_coords = projected_points - np.array([x0, y0, z0])
    u_coords = np.dot(relative_coords, basis1)
    v_coords = np.dot(relative_coords, basis2)

    # Distance to nearest edge line (considering the rectangle as a combination of four lines)
    edge_distances = np.minimum(np.abs(u_coords - width / 2), np.abs(u_coords + width / 2)) + \
                     np.minimum(np.abs(v_coords - height / 2), np.abs(v_coords + height / 2))

    return edge_distances

def fit_rectangle(points):
    x0, y0, z0 = np.mean(points, axis=0)
    initial_normal = np.array([0, 0, 1])  # Initial guess for plane normal
    width_guess = np.max(points[:, 0]) - np.min(points[:, 0])
    height_guess = np.max(points[:, 1]) - np.min(points[:, 1])
    initial_guess = [x0, y0, z0, *initial_normal, width_guess, height_guess]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x  # [x0, y0, z0, nx, ny, nz, width, height]

def ransac_rectangle(points, threshold, iterations):
    densities = calculate_density(points)
    
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        print(f"Iteration: {i + 1}")
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_rectangle(sample)
        x0, y0, z0, nx, ny, nz, width, height = params

        plane_normal = np.array([nx, ny, nz])
        plane_normal /= np.linalg.norm(plane_normal)

        # Define orthogonal vectors on the plane
        basis1 = np.cross(plane_normal, np.array([1, 0, 0]))
        if np.linalg.norm(basis1) < 1e-6:
            basis1 = np.cross(plane_normal, np.array([0, 1, 0]))
        basis1 /= np.linalg.norm(basis1)
        basis2 = np.cross(plane_normal, basis1)

        inliers = []

        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            distance_to_plane = np.dot(vector_to_point, plane_normal)

            # Project points onto the plane
            projection = vector_to_point - distance_to_plane * plane_normal
            projected_point = projection + np.array([x0, y0, z0])

            # Transform into local rectangle frame
            relative_coords = projected_point - np.array([x0, y0, z0])
            u_coords = np.dot(relative_coords, basis1)
            v_coords = np.dot(relative_coords, basis2)

            # Calculate distance to edges
            edge_distance = np.minimum(np.abs(u_coords - width / 2), np.abs(u_coords + width / 2)) + \
                            np.minimum(np.abs(v_coords - height / 2), np.abs(v_coords + height / 2))

            # Check if the point is close enough to the edges
            if edge_distance < threshold:
                inliers.append((point, idx))

        total_score = len(inliers)

        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [28]:
import numpy as np

def sample_rectangle(points):
    """
    Randomly samples four points from the input data.
    """
    return points[np.random.choice(points.shape[0], 4, replace=False)]

def fit_plane(points):
    """
    Fit a plane to the given points using SVD.
    Returns the plane parameters (normal vector and offset).
    """
    centroid = np.mean(points, axis=0)
    _, _, vh = np.linalg.svd(points - centroid)
    normal = vh[2, :]
    d = -np.dot(normal, centroid)
    return np.append(normal, d)

def project_points_to_plane(points, plane_params):
    """
    Projects points onto a given plane defined by plane parameters.
    """
    normal = plane_params[:3]
    d = plane_params[3]
    normal = normal / np.linalg.norm(normal)
    projected_points = points - (np.dot(points, normal) + d)[:, np.newaxis] * normal
    return projected_points

def construct_rectangle_from_points(points):
    """
    Constructs a rectangle from four sampled points by fitting a plane, defining edges,
    and ensuring perpendicularity of the constructed rectangle.
    """
    # Step 1: Fit the plane to the sampled points
    plane_params = fit_plane(points)
    projected_points = project_points_to_plane(points, plane_params)

    # Step 2: Find the center of the rectangle
    center = np.mean(projected_points, axis=0)

    # Step 3: Define two edge vectors using SVD, automatically adjusting to make them perpendicular
    _, _, vh = np.linalg.svd(projected_points - center)
    edge1 = vh[0, :]  # Main edge direction
    edge2 = vh[1, :]  # Perpendicular direction

    # Step 4: Calculate the extents of the rectangle (width and height)
    u_coords = np.dot(projected_points - center, edge1)
    v_coords = np.dot(projected_points - center, edge2)
    width = np.max(u_coords) - np.min(u_coords)
    height = np.max(v_coords) - np.min(v_coords)

    # Re-adjust edges to ensure perpendicularity if needed (edge2 is already perpendicular due to SVD)
    edge1 /= np.linalg.norm(edge1)
    edge2 /= np.linalg.norm(edge2)

    return plane_params, center, edge1, edge2, width, height

def score_rectangle_fit(points, plane_params, center, edge1, edge2, width, height, tolerance):
    """
    Scores the fit of the points within or near the rectangle's boundaries.
    """
    projected_points = project_points_to_plane(points, plane_params)
    relative_coords = projected_points - center
    u_coords = np.dot(relative_coords, edge1)
    v_coords = np.dot(relative_coords, edge2)

    # Check if points are within or close to the rectangle bounds
    in_bounds = (np.abs(u_coords) <= width / 2 + tolerance) & (np.abs(v_coords) <= height / 2 + tolerance)
    inliers = points[in_bounds]
    return len(inliers), inliers

def ransac_rectangle(points, threshold, iterations, tolerance=0.01):
    """
    RANSAC algorithm to fit a finite rectangle to 3D points.
    """
    best_inliers = []
    best_params = None

    for i in range(iterations):
        print(f"Iteration {i+1}")
        # Step 1: Sample four points to define a potential rectangle
        sampled_points = sample_rectangle(points)

        # Step 2: Construct a proper rectangle using the four points
        plane_params, center, edge1, edge2, width, height = construct_rectangle_from_points(sampled_points)

        # Step 3: Score the fit based on proximity to the rectangle's edges
        score, inliers = score_rectangle_fit(points, plane_params, center, edge1, edge2, width, height, tolerance)

        # Update the best model if this one has more inliers
        if score > len(best_inliers):
            best_inliers = inliers
            best_params = (center, edge1, edge2, width, height)

    print(f"Best Score: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [94]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, LineString

def fit_plane(points):
    """
    Fits a plane to the given points using SVD.
    Returns the plane parameters (normal vector and offset).
    """
    centroid = np.mean(points, axis=0)
    _, _, vh = np.linalg.svd(points - centroid)
    normal = vh[2, :]
    d = -np.dot(normal, centroid)
    return np.append(normal, d)

def project_points_onto_plane(points, plane_params):
    """
    Projects points onto the fitted plane.
    """
    normal = plane_params[:3]
    d = plane_params[3]
    normal = normal / np.linalg.norm(normal)
    projected_points = points - (np.dot(points, normal) + d)[:, np.newaxis] * normal
    return projected_points

def filter_connected_inliers(inliers, connectivity_threshold):
    """
    Filters inliers to keep only the largest connected group of points based on nearest neighbor connectivity.
    
    Parameters:
        inliers (ndarray): Array of inlier points.
        connectivity_threshold (float): Maximum distance to consider points as connected neighbors.

    Returns:
        ndarray: Filtered inliers that form the largest connected component.
    """
    if len(inliers) == 0:
        return np.array([])

    # Fit nearest neighbors
    nbrs = NearestNeighbors(radius=connectivity_threshold).fit(inliers)
    distances, indices = nbrs.radius_neighbors(inliers)

    # Use a visited array to keep track of connected points
    visited = np.zeros(len(inliers), dtype=bool)
    connected_components = []

    # Explore connected components
    for i in range(len(inliers)):
        if not visited[i]:
            stack = [i]
            component = []

            # Depth-first search to find all connected points
            while stack:
                idx = stack.pop()
                if not visited[idx]:
                    visited[idx] = True
                    component.append(inliers[idx])
                    neighbors = indices[idx]
                    stack.extend(neighbors)

            # Store the component if it's large enough
            connected_components.append(component)

    # Find the largest connected component
    largest_component = max(connected_components, key=len)
    return np.array(largest_component)

def simplify_polygon(points, tolerance):
    """
    Simplifies the boundary of the convex hull using the Douglas-Peucker algorithm to find corners.

    Parameters:
        points (ndarray): Convex hull points in 2D.
        tolerance (float): Tolerance for the simplification; lower values retain more detail.

    Returns:
        ndarray: Simplified corner points of the polygon.
    """
    if len(points) < 3:
        return points

    # Create a shapely polygon and simplify it
    polygon = Polygon(points)
    simplified = polygon.simplify(tolerance, preserve_topology=True)

    # Extract the simplified corner points
    return np.array(simplified.exterior.coords[:-1])  # Exclude the repeated last point

def ransac_plane(points, threshold, iterations, connectivity_threshold, simplification_tolerance):
    """
    RANSAC algorithm to fit a plane to the 3D points, ensuring inliers form a connected set.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        threshold (float): Distance threshold for inlier consideration.
        iterations (int): Number of RANSAC iterations.
        connectivity_threshold (float): Maximum distance to consider inliers as part of the same connected group.
        simplification_tolerance (float): Tolerance for simplifying the convex hull to find corners.

    Returns:
        best_params (ndarray): Best plane parameters found [normal_x, normal_y, normal_z, d].
        best_inliers (ndarray): Array of inlier points for the best model.
        corners (ndarray): Simplified corner points of the 2D plane.
    """
    best_inliers = []
    best_params = None
    best_corners = []

    for i in range(iterations):
        print(f"Iteration: {i + 1}")

        # Randomly sample 3 points to define a plane
        sample = points[np.random.choice(points.shape[0], 3, replace=False)]
        params = fit_plane(sample)
        inliers = []

        # Calculate inliers based on the distance to the plane
        for point in points:
            distance = np.abs(np.dot(params[:3], point) + params[3]) / np.linalg.norm(params[:3])
            if distance < threshold:
                inliers.append(point)

        # Convert inliers to numpy array
        inliers = np.array(inliers)

        # Check connectivity between inliers to ensure a continuous group
        if len(inliers) > 0:
            connected_inliers = filter_connected_inliers(inliers, connectivity_threshold)

            # Update the best model if this one has more connected inliers
            if len(connected_inliers) > len(best_inliers):
                best_inliers = connected_inliers
                best_params = params

                # Project inliers to 2D on the plane
                projected_inliers = project_points_onto_plane(best_inliers, best_params)

                # Find the convex hull and simplify it to extract corners
                if len(projected_inliers) > 3:
                    hull = ConvexHull(projected_inliers[:, :2])  # Use only the x and y coordinates
                    hull_points = projected_inliers[hull.vertices, :2]
                    corners = simplify_polygon(hull_points, simplification_tolerance)
                    best_corners = corners

    print(f"Best Score (Inliers): {len(best_inliers)}")
    return best_params, np.array(best_inliers), best_corners

In [146]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def fit_plane(points):
    """
    Fits a plane to the given points using SVD.
    Returns the plane parameters (normal vector and offset).
    """
    centroid = np.mean(points, axis=0)
    _, _, vh = np.linalg.svd(points - centroid)
    normal = vh[2, :]
    d = -np.dot(normal, centroid)
    return np.append(normal, d)

def project_points_onto_plane(points, plane_params):
    """
    Projects points onto the fitted plane.
    """
    normal = plane_params[:3]
    d = plane_params[3]
    normal = normal / np.linalg.norm(normal)
    projected_points = points - (np.dot(points, normal) + d)[:, np.newaxis] * normal
    return projected_points

def filter_connected_inliers(inliers, connectivity_threshold):
    """
    Filters inliers to keep only the largest connected group of points based on nearest neighbor connectivity.
    """
    if len(inliers) == 0:
        return np.array([])

    nbrs = NearestNeighbors(radius=connectivity_threshold).fit(inliers)
    distances, indices = nbrs.radius_neighbors(inliers)

    visited = np.zeros(len(inliers), dtype=bool)
    connected_components = []

    for i in range(len(inliers)):
        if not visited[i]:
            stack = [i]
            component = []

            while stack:
                idx = stack.pop()
                if not visited[idx]:
                    visited[idx] = True
                    component.append(inliers[idx])
                    neighbors = indices[idx]
                    stack.extend(neighbors)

            connected_components.append(component)

    largest_component = max(connected_components, key=len)
    return np.array(largest_component)

def find_bounding_box_pca(points_2d):
    """
    Finds the minimum bounding rectangle of a set of 2D points using PCA alignment.

    Parameters:
        points_2d (ndarray): 2D points projected onto a plane.

    Returns:
        ndarray: Four corner points of the bounding rectangle.
    """
    pca = PCA(n_components=2)
    points_aligned = pca.fit_transform(points_2d)

    # Find the min and max corners in the aligned space
    min_x, min_y = np.min(points_aligned, axis=0)
    max_x, max_y = np.max(points_aligned, axis=0)

    # Define the rectangle in the PCA space
    corners_pca = np.array([
        [min_x, min_y],
        [max_x, min_y],
        [max_x, max_y],
        [min_x, max_y]
    ])

    # Reproject the corners back to the original 2D space
    corners_2d = pca.inverse_transform(corners_pca)
    return corners_2d

def reproject_corners_to_3d(corners_2d, plane_params):
    """
    Reprojects 2D corners back to 3D using the plane's orientation.

    Parameters:
        corners_2d (ndarray): 2D corners of the bounding rectangle.
        plane_params (ndarray): Parameters of the fitted plane.

    Returns:
        ndarray: 3D coordinates of the corners on the plane.
    """
    normal = plane_params[:3]
    normal = normal / np.linalg.norm(normal)
    d = plane_params[3]

    corners_3d = []
    for corner in corners_2d:
        corner_3d = np.array([corner[0], corner[1], 0])
        distance_to_plane = -(np.dot(corner_3d, normal) + d)
        point_3d = corner_3d + distance_to_plane * normal
        corners_3d.append(point_3d)

    return np.array(corners_3d)

def ransac_rectangle(points, threshold, iterations, connectivity_threshold):
    """
    RANSAC algorithm to fit a plane to the 3D points and detect a rectangular shape.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        threshold (float): Distance threshold for inlier consideration.
        iterations (int): Number of RANSAC iterations.
        connectivity_threshold (float): Maximum distance to consider inliers as part of the same connected group.

    Returns:
        best_params (ndarray): Best plane parameters found [normal_x, normal_y, normal_z, d].
        best_inliers (ndarray): Array of inlier points for the best model.
        rectangle_corners (ndarray): 3D coordinates of the detected rectangle's corners.
    """
    best_inliers = []
    best_params = None
    rectangle_corners = []

    for i in range(iterations):
        print(f"Iteration: {i + 1}")

        sample = points[np.random.choice(points.shape[0], 3, replace=False)]
        params = fit_plane(sample)
        inliers = []

        for point in points:
            distance = np.abs(np.dot(params[:3], point) + params[3]) / np.linalg.norm(params[:3])
            if distance < threshold:
                inliers.append(point)

        inliers = np.array(inliers)

        if len(inliers) > 0:
            connected_inliers = filter_connected_inliers(inliers, connectivity_threshold)

            if len(connected_inliers) > len(best_inliers):
                best_inliers = connected_inliers
                best_params = params

                projected_inliers = project_points_onto_plane(best_inliers, best_params)

                if len(projected_inliers) > 3:
                    corners_2d = find_bounding_box_pca(projected_inliers[:, :2])
                    rectangle_corners = reproject_corners_to_3d(corners_2d, best_params)

    print(f"Best Score (Inliers): {len(best_inliers)}")
    return best_params, np.array(best_inliers), rectangle_corners

In [147]:



plane_params, plane_inliers , edges= ransac_rectangle(points, iterations=100, threshold = 0.001, connectivity_threshold =0.01)



Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [ ]:
plane_cloud = o3d.geometry.PointCloud()
plane_cloud.points = o3d.utility.Vector3dVector(plane_inliers)
plane_cloud.paint_uniform_color([0,1,0])

edge_cloud = o3d.geometry.PointCloud()
edge_cloud.points = o3d.utility.Vector3dVector(edges)
edge_cloud.paint_uniform_color([1,0,0])


downpcd.paint_uniform_color([0,0,0])

o3d.visualization.draw_geometries([downpcd, plane_cloud, edge_cloud])